In [1]:

from copy import deepcopy

import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm

In [2]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *

In [3]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def train_sparse_recursive(model, train_loader, val_loader, num_epochs, metric, edge_replacement_func=None,
                           window_size=2, threshold=0.10):
    optimizer = optim.Adam(model.parameters(), lr=5e-5)
    criterion = nn.CrossEntropyLoss()

    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for inputs, targets in tqdm(train_loader):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0
        all_targets = []
        all_preds = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_targets.extend(targets.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = accuracy_score(all_targets, all_preds)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        new_l = {}
        # if edge_replacement_func and epoch % 8 == 0 and epoch != 0:
        #     new_l = edge_replacement_func(model, optimizer, val_loader, metric)
        val_losses.append(val_loss)
        if len(val_losses) > window_size:
            recent_changes = [abs(val_losses[i] - val_losses[i - 1]) for i in range(-window_size, 0)]
            avg_change = sum(recent_changes) / window_size
            if avg_change < threshold:
                new_l = edge_replacement_func(model, optimizer, val_loader, metric)

                if new_l["len_choose"] == 0:
                    break

        wandb.log({'val loss': val_loss, 'val accuracy': val_accuracy, 'train loss': train_loss} | new_l)


def edge_replacement_func_new_layer(model, optim, val_loader, metric):
    layer = model.fc1
    ef = EdgeFinder(metric, val_loader, device)
    vals = ef.calculate_edge_metric_for_dataloader(model)
    print(f"{len(vals)=} {max(vals)=}  {sum(vals)=} {min(vals)=}")
    chosen_edges = ef.choose_edges_threshold(model, 0.15)
    print("choose:", chosen_edges, len(chosen_edges[0]))
    layer.replace_many(*chosen_edges)

    if layer.embed_linears:
        optim.add_param_group({'params': layer.embed_linears[-1].weight_values})
    else:
        print("empty metric")
        dummy_param = torch.zeros_like(layer.weight_values)
        optim.add_param_group({'params': dummy_param})

    return {'max': max(vals), 'sum': sum(vals), 'len': len(vals), 'len_choose': len(chosen_edges[0])}

In [5]:
# Define the model
class SimpleFCN(nn.Module):
    def __init__(self, input_size=28 * 28):
        super(SimpleFCN, self).__init__()
        self.fc1 = nn.Linear(input_size, 10)

    def forward(self, x):
        x = self.fc1(x)
        return x

In [6]:
# Dataset and Dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Load dataset and split into train/validation sets
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [7]:
label_dict = {}
for i in val_dataset:
    if i[1] not in label_dict.keys():
        label_dict[i[1]] = 0
    else:
        label_dict[i[1]] += 1
label_dict

{4: 1179,
 7: 1285,
 9: 1139,
 5: 1121,
 3: 1193,
 6: 1191,
 8: 1150,
 0: 1225,
 1: 1353,
 2: 1154}

In [8]:
criterion = nn.CrossEntropyLoss()
metrics = [
    MagnitudeL2Metric(criterion),
    SNIPMetric(criterion),
    GradientMeanEdgeMetric(criterion),
    PerturbationSensitivityEdgeMetric(criterion),
]
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model)
sparse_linear = deepcopy(sparse_model.fc1)
sparse_model.fc1.weight_indices

tensor([[  0,   0,   0,  ...,   9,   9,   9],
        [  0,   1,   2,  ..., 781, 782, 783]])

In [9]:
sparse_model.fc1.weight_indices[:, :50]

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]])

In [10]:
# ef = EdgeFinder(metrics[0], val_loader, device)
# print(ef.choose_edges_threshold(sparse_model, 0.3))

# raise "e"

In [11]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fedornigretuk. Use `wandb login --relogin` to force relogin


True

In [12]:
run = wandb.init(
    project="self-expanding-nets",
    name=f"replace=(auto epoch, threshold 0.15), lr=5e-5, magnetic l2 metric",
)

In [13]:
train_sparse_recursive(sparse_model, train_loader, val_loader, 64, metrics[0],
                       edge_replacement_func=edge_replacement_func_new_layer)

100%|██████████| 750/750 [00:10<00:00, 74.65it/s]


Epoch 1/64, Train Loss: 1.7941, Val Loss: 1.3948, Val Accuracy: 0.7672


100%|██████████| 750/750 [00:09<00:00, 78.61it/s]


Epoch 2/64, Train Loss: 1.1640, Val Loss: 0.9907, Val Accuracy: 0.8234


100%|██████████| 750/750 [00:09<00:00, 83.29it/s]


...

Epoch 40/64, Train Loss: 0.9739, Val Loss: 0.9684, Val Accuracy: 0.7842
len(vals)=7840 max(vals)=tensor(1., grad_fn=<UnbindBackward0>)  sum(vals)=tensor(109.3760, grad_fn=<AddBackward0>) min(vals)=tensor(0., grad_fn=<UnbindBackward0>)
choose: tensor([[   2,    6,    5,    0,    1,    2,    2,    5,    7,    8,    9,    9,
            9,    2,    2,    4,    6,    6,    6,    9,    0,    1,    1,    2,
            4,    6,    6,    9,    3,    8,    2,    4,    6,    7,    7,    1,
            5,    7,    8,    9,    9,    6,    8,    7,    0,    8,    6,    7,
            0,    0,    1,    6,    1,    1,    7,    1,    8,    4,    2,    1,
            8,    5,    1,    0,    0],
        [7091, 7141, 7202, 7236, 7237, 7238, 7239, 7241, 7243, 7244, 7245, 7246,
         7247, 7249, 7250, 7253, 7256, 7257, 7258, 7261, 7263, 7265, 7266, 7268,
         7269, 7270, 7271, 7273, 7274, 7275, 7276, 7277, 7278, 7279, 7280, 7287,
         7288, 7289, 7290, 7291, 7293, 7294, 7295, 7296, 7298, 7300, 

100%|██████████| 750/750 [01:13<00:00, 10.22it/s]


Epoch 41/64, Train Loss: 0.9554, Val Loss: 0.9518, Val Accuracy: 0.7847
len(vals)=7840 max(vals)=tensor(1., grad_fn=<UnbindBackward0>)  sum(vals)=tensor(90.2769, grad_fn=<AddBackward0>) min(vals)=tensor(0., grad_fn=<UnbindBackward0>)
choose: tensor([[   2,    6,    1,    2,    5,    8,    9,    2,    4,    6,    6,    6,
            9,    1,    2,    6,    6,    8,    4,    1,    7,    8,    6,    8,
            7,    0,    0,    1,    6,    1,    1,    7,    1,    4,    2,    8,
            1,    0,    0],
        [7328, 7329, 7332, 7334, 7335, 7337, 7340, 7341, 7343, 7344, 7345, 7346,
         7347, 7349, 7351, 7353, 7354, 7357, 7359, 7363, 7365, 7366, 7369, 7373,
         7375, 7376, 7377, 7378, 7379, 7380, 7381, 7382, 7383, 7385, 7386, 7388,
         7390, 7391, 7392]]) 39


 64%|██████▎   | 478/750 [00:43<00:24, 10.99it/s]


KeyboardInterrupt: 